Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [67]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import math

In [68]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [69]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [70]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [71]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [72]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.868744
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.827591
Minibatch accuracy: 25.0%
Validation accuracy: 46.4%
Minibatch loss at step 100: 1.068059
Minibatch accuracy: 62.5%
Validation accuracy: 67.3%
Minibatch loss at step 150: 0.565447
Minibatch accuracy: 75.0%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.909988
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 250: 1.537019
Minibatch accuracy: 56.2%
Validation accuracy: 77.3%
Minibatch loss at step 300: 0.389098
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.395636
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.259601
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.959288
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 500: 0.695759
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [73]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # First Convolutional Layer
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # Second Convolutional Layer
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [74]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.734054
Minibatch accuracy: 6.2%
Validation accuracy: 12.1%
Minibatch loss at step 50: 2.147985
Minibatch accuracy: 31.2%
Validation accuracy: 32.6%
Minibatch loss at step 100: 1.070772
Minibatch accuracy: 68.8%
Validation accuracy: 57.3%
Minibatch loss at step 150: 0.543597
Minibatch accuracy: 81.2%
Validation accuracy: 71.3%
Minibatch loss at step 200: 0.868657
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.289828
Minibatch accuracy: 62.5%
Validation accuracy: 77.9%
Minibatch loss at step 300: 0.314129
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 350: 0.593264
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.294904
Minibatch accuracy: 93.8%
Validation accuracy: 80.9%
Minibatch loss at step 450: 0.813232
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.470949
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [123]:
batch_size = 128
patch_size = 5
depth_1 = 16
depth_2 = 32
depth_3 = 64
num_hidden_1 = 64
num_hidden_2 = 32

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))

  layer3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth_2, depth_3], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth_3]))  

  layer4_weights = tf.Variable(tf.truncated_normal([1024, num_hidden_1], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_1]))

  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))

  layer6_weights = tf.Variable(tf.truncated_normal([num_hidden_2, num_labels], stddev=0.1))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
                   
  # Dropout
  keep_prob = tf.placeholder(tf.float32)
  
  # Model.
  def model(data):
    # First Convolutional Layer
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.max_pool(hidden_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # Second Convolutional Layer
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # Third Convolutional Layer
    conv_3 = tf.nn.conv2d(pool_2, layer3_weights, [1, 1, 1, 1], padding='SAME')
    hidden_3 = tf.nn.relu(conv_3 + layer3_biases)
    pool_3 = tf.nn.max_pool(hidden_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # First Densely Connected Layer 
    shape = pool_3.get_shape().as_list()
    reshape = tf.reshape(pool_3, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_4 = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    
    # Second Densely Connected Layer 
    hidden_5 = tf.nn.relu(tf.matmul(hidden_4, layer5_weights) + layer5_biases)
    
    # Readout Layer
    #hidden_drop = tf.nn.dropout(hidden_3, keep_prob)
    densely = tf.matmul(hidden_5, layer6_weights) + layer6_biases 
    return densely
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.12, global_step, 5, 0.999)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

reshape (128, 1024)
layer4_weights (1024, 64)
layer4_biases (64,)
reshape (10000, 1024)
layer4_weights (1024, 64)
layer4_biases (64,)
reshape (10000, 1024)
layer4_weights (1024, 64)
layer4_biases (64,)


In [124]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    #feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.75}
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.035683
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.218642
Minibatch accuracy: 15.6%
Validation accuracy: 15.4%
Minibatch loss at step 100: 1.471066
Minibatch accuracy: 40.6%
Validation accuracy: 43.3%
Minibatch loss at step 150: 1.055316
Minibatch accuracy: 68.8%
Validation accuracy: 76.9%
Minibatch loss at step 200: 0.747598
Minibatch accuracy: 79.7%
Validation accuracy: 78.5%
Minibatch loss at step 250: 0.397178
Minibatch accuracy: 93.0%
Validation accuracy: 82.1%
Minibatch loss at step 300: 0.624370
Minibatch accuracy: 82.8%
Validation accuracy: 82.4%
Minibatch loss at step 350: 0.375912
Minibatch accuracy: 89.8%
Validation accuracy: 83.7%
Minibatch loss at step 400: 0.534365
Minibatch accuracy: 83.6%
Validation accuracy: 84.1%
Minibatch loss at step 450: 0.398622
Minibatch accuracy: 88.3%
Validation accuracy: 85.2%
Minibatch loss at step 500: 0.378862
Minibatch accuracy: 90.6%
Validation accuracy: 85.9%
M